# Day 7: Model Explainability for Fraud Detection

**Interpreting ML Models for Trust and Compliance**

## Overview
- **Objective**: Understand why models make specific predictions
- **Techniques**: SHAP values, LIME, Feature Importance
- **Importance**: Regulatory compliance (GDPR, Fair Credit Reporting Act)

## What You'll Learn
1. **Global Explainability**: Overall feature importance
2. **Local Explainability**: Individual prediction explanations
3. **SHAP Values**: Game theory-based attribution
4. **LIME**: Local surrogate models

---

## 1. Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported!")
print("\nNote: For full SHAP and LIME functionality, install:")
print("  pip install shap lime")

## 2. Generate Sample Fraud Data

In [ ]:
# Generate transaction data
np.random.seed(42)

n_samples = 10000
fraud_ratio = 0.05

# Features
df = pd.DataFrame({
    'transaction_amount': np.random.lognormal(4, 1.5, n_samples),
    'hour_of_day': np.random.randint(0, 24, n_samples),
    'day_of_week': np.random.randint(0, 7, n_samples),
    'age': np.random.randint(18, 80, n_samples),
    'transaction_count_24h': np.random.randint(0, 20, n_samples),
    'avg_amount_30d': np.random.lognormal(3.5, 1, n_samples),
    'merchant_risk_score': np.random.uniform(0, 1, n_samples),
    'distance_from_home': np.random.exponential(50, n_samples),
    'is_international': np.random.choice([0, 1], n_samples, p=[0.9, 0.1]),
    'card_present': np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
})

# Generate labels based on rules (with some noise)
fraud_score = (
    (df['transaction_amount'] > 5000) * 2 +
    (df['hour_of_day'].isin([0, 1, 2, 3, 4, 22, 23])) * 1.5 +
    (df['transaction_count_24h'] > 10) * 1 +
    (df['merchant_risk_score'] > 0.7) * 1.5 +
    (df['distance_from_home'] > 100) * 1 +
    (df['is_international'] == 1) * 0.5 -
    (df['card_present'] == 1) * 0.5
)

# Convert to binary with threshold
threshold = np.percentile(fraud_score, 100 * (1 - fraud_ratio))
df['is_fraud'] = (fraud_score >= threshold).astype(int)

print(f"Dataset shape: {df.shape}")
print(f"Fraud rate: {df['is_fraud'].mean()*100:.2f}%")
print(f"\nSample data:")
print(df.head())

## 3. Train Models

We'll train multiple models to compare their explainability

In [ ]:
# Prepare data
feature_cols = [c for c in df.columns if c != 'is_fraud']
X = df[feature_cols].values
y = df['is_fraud'].values

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_prob)
    print(f"{name:.<30} AUC: {auc:.3f}")

## 4. Global Explainability - Feature Importance

In [ ]:
# Plot feature importance for tree-based models
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Random Forest
rf_importance = models['Random Forest'].feature_importances_
indices_rf = np.argsort(rf_importance)[::-1]
axes[0].barh(range(len(feature_cols)), rf_importance[indices_rf], color='steelblue')
axes[0].set_yticks(range(len(feature_cols)))
axes[0].set_yticklabels([feature_cols[i] for i in indices_rf])
axes[0].set_xlabel('Importance', fontsize=12)
axes[0].set_title('Random Forest Feature Importance', fontsize=14)
axes[0].invert_yaxis()

# Gradient Boosting
gb_importance = models['Gradient Boosting'].feature_importances_
indices_gb = np.argsort(gb_importance)[::-1]
axes[1].barh(range(len(feature_cols)), gb_importance[indices_gb], color='coral')
axes[1].set_yticks(range(len(feature_cols)))
axes[1].set_yticklabels([feature_cols[i] for i in indices_gb])
axes[1].set_xlabel('Importance', fontsize=12)
axes[1].set_title('Gradient Boosting Feature Importance', fontsize=14)
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("\nTop 5 Features (Random Forest):")
for i in indices_rf[:5]:
    print(f"  {feature_cols[i]:.<30} {rf_importance[i]:.3f}")

## 5. Logistic Regression - Coefficient Analysis

In [ ]:
# Logistic regression coefficients (with direction)
lr_model = models['Logistic Regression']
coefs = lr_model.coef_[0]

# Sort by absolute value
indices_lr = np.argsort(np.abs(coefs))[::-1]

# Plot
plt.figure(figsize=(12, 6))
colors = ['red' if coefs[i] > 0 else 'blue' for i in range(len(coefs))]
plt.barh(range(len(feature_cols)), coefs[indices_lr], color=colors, alpha=0.7)
plt.yticks(range(len(feature_cols)), [feature_cols[i] for i in indices_lr])
plt.xlabel('Coefficient Value', fontsize=12)
plt.title('Logistic Regression Coefficients\n(Red=Increases Fraud Risk, Blue=Decreases)', fontsize=14)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nLogistic Regression Interpretation:")
print("  Positive coefficients: INCREASE fraud probability")
print("  Negative coefficients: DECREASE fraud probability")
print("\nTop 5 Features (by absolute coefficient):")
for i in indices_lr[:5]:
    direction = "↑" if coefs[i] > 0 else "↓"
    print(f"  {feature_cols[i]:.<30} {direction} {abs(coefs[i]):.3f}")

## 6. SHAP Values (Simplified Implementation)

In [ ]:
# Simplified SHAP-like value calculation
# (Full SHAP requires shap library)

def calculate_shap_like_values(model, X, feature_names):
    """
    Simplified SHAP value calculation using feature permutation.
    This demonstrates the concept without requiring the shap library.
    """
    # Baseline prediction (mean)
    baseline_pred = model.predict_proba(X)[:, 1].mean()
    
    shap_values = []
    
    for i in range(X.shape[1]):
        # Permute feature i
        X_permuted = X.copy()
        np.random.shuffle(X_permuted[:, i])
        
        # Prediction with permuted feature
        permuted_pred = model.predict_proba(X_permuted)[:, 1].mean()
        
        # SHAP value = baseline - permuted
        shap_value = baseline_pred - permuted_pred
        shap_values.append(shap_value)
    
    return np.array(shap_values)

# Calculate for a random sample
sample_idx = np.random.choice(len(X_test))
sample = X_test[sample_idx:sample_idx+1]

# Get prediction
lr_model = models['Logistic Regression']
pred_prob = lr_model.predict_proba(sample)[0, 1]

# Calculate SHAP-like values
shap_values = calculate_shap_like_values(lr_model, X_test, feature_cols)

# Plot
plt.figure(figsize=(12, 6))
colors = ['red' if v > 0 else 'blue' for v in shap_values]
plt.barh(range(len(feature_cols)), shap_values, color=colors, alpha=0.7)
plt.yticks(range(len(feature_cols)), feature_cols)
plt.xlabel('SHAP Value (Impact on Prediction)', fontsize=12)
plt.title(f'SHAP Values for Prediction (Fraud Prob={pred_prob:.3f})\n(Red=Pushes Toward Fraud, Blue=Pushes Away)', fontsize=14)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nSHAP Value Interpretation:")
print("  Positive values: Feature pushes prediction toward FRAUD")
print("  Negative values: Feature pushes prediction toward LEGITIMATE")
print("\nActual label:", y_test[sample_idx])
print(f"Predicted fraud probability: {pred_prob:.3f}")

## 7. Partial Dependence Plots

In [ ]:
# Partial dependence for top features
from sklearn.inspection import PartialDependenceDisplay

# Get top 3 features
top_features = [feature_cols[i] for i in indices_rf[:3]]

# Create PDP
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, feat in enumerate(top_features):
    feat_idx = feature_cols.index(feat)
    
    # Calculate partial dependence
    feature_values = np.percentile(X_test[:, feat_idx], np.linspace(0, 100, 50))
    avg_predictions = []
    
    for val in feature_values:
        X_temp = X_test.copy()
        X_temp[:, feat_idx] = val
        avg_pred = models['Random Forest'].predict_proba(X_temp)[:, 1].mean()
        avg_predictions.append(avg_pred)
    
    axes[idx].plot(feature_values, avg_predictions, 'o-', linewidth=2, markersize=4)
    axes[idx].set_xlabel(feat, fontsize=11)
    axes[idx].set_ylabel('Average Fraud Probability', fontsize=11)
    axes[idx].set_title(f'Partial Dependence: {feat}', fontsize=12)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nPartial Dependence Interpretation:")
print("  Shows how average fraud probability changes with feature values")
print("  Helps understand the marginal effect of each feature")

## 8. LIME-style Local Explanation (Simplified)

In [ ]:
# LIME-like local explanation
def explain_local_prediction(model, sample, feature_names, num_samples=1000):
    """
    Generate local explanation using perturbed samples.
    This mimics LIME's approach.
    """
    # Generate perturbed samples around the instance
    X_perturbed = np.tile(sample, (num_samples, 1))
    
    # Add noise proportional to feature std
    for i in range(sample.shape[1]):
        feature_std = X_test[:, i].std()
        noise = np.random.normal(0, feature_std * 0.5, num_samples)
        X_perturbed[:, i] += noise
    
    # Get predictions
    predictions = model.predict_proba(X_perturbed)[:, 1]
    
    # Calculate weights (distance from original sample)
    distances = np.linalg.norm(X_perturbed - sample, axis=1)
    kernel_width = distances.std()
    weights = np.sqrt(np.exp(-(distances ** 2) / (kernel_width ** 2)))
    
    # Fit local linear model
    from sklearn.linear_model import Ridge
    local_model = Ridge(alpha=1.0)
    local_model.fit(X_perturbed, predictions, sample_weight=weights)
    
    return local_model.coef_

# Get a fraud sample
fraud_indices = np.where(y_test == 1)[0]
if len(fraud_indices) > 0:
    sample_idx = fraud_indices[0]
    sample = X_test[sample_idx:sample_idx+1]
    
    # Get explanation
    lr_model = models['Logistic Regression']
    local_coefs = explain_local_prediction(lr_model, sample, feature_cols)
    
    # Plot
    plt.figure(figsize=(12, 6))
    colors = ['red' if c > 0 else 'blue' for c in local_coefs]
    plt.barh(range(len(feature_cols)), local_coefs, color=colors, alpha=0.7)
    plt.yticks(range(len(feature_cols)), feature_cols)
    plt.xlabel('Local Feature Importance', fontsize=12)
    plt.title(f'LIME-style Local Explanation\n(Red=Contributed to Fraud, Blue=Contributed to Legitimate)', fontsize=14)
    plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print(f"\nActual label: {y_test[sample_idx]} (Fraud)")
    print(f"Predicted fraud prob: {lr_model.predict_proba(sample)[0, 1]:.3f}")
else:
    print("No fraud samples in test set!")

## 9. Summary

### Model Explainability Techniques:

1. **Feature Importance (Global)**:
   - Tree-based: Mean decrease in impurity
   - Linear: Coefficient magnitude
   - Shows which features matter most overall

2. **SHAP Values (Local + Global)**:
   - Game theory-based
   - Fair attribution across features
   - Explains individual predictions

3. **Partial Dependence (Global)**:
   - Shows marginal effect of features
   - Helps understand feature relationships

4. **LIME (Local)**:
   - Local surrogate model
   - Explains individual predictions
   - Model-agnostic

### Key Takeaways:

- ✅ **Interpretability is crucial** for trust and compliance
- ✅ **Different models** have different explainability needs
- ✅ **Global explanations** show overall patterns
- ✅ **Local explanations** explain individual decisions

### Regulatory Requirements:

- **GDPR**: Right to explanation for automated decisions
- **FCRA**: Adverse action requires explanation
- **Model Risk Management**: Regulators require model validation

### Best Practices:

1. **Use interpretable models when possible** (logistic regression, decision trees)
2. **Post-hoc explanations** for complex models (SHAP, LIME)
3. **Document** model behavior and feature importance
4. **Validate** explanations make domain sense

### Next Steps:
→ **Day 8**: Federated Averaging (FedAvg)

---

**📁 Project Location**: `01_fraud_detection_core/fraud_model_explainability/`